In [ ]:
#@title Copyright 2020 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/Manifest_image_upload_demo.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/Manifest_image_upload_demo.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Uploading an image from tiles using a manifest

This notebook demonstrates uploading a set of image tiles into a single asset using a manifest file. See [this doc](https://developers.google.com/earth-engine/image_manifest) for more details about manifest upload using the Earth Engine command line tool.

First, authenticate with Google Cloud, so you can access Cloud Storage buckets.

In [ ]:
from google.colab import auth
auth.authenticate_user()

## Upload source images to Cloud Storage

The source images must exist in a Google Cloud Storage bucket. For this tutorial we have provided six [land cover image tiles](https://data-starcloud.pcl.ac.cn/resource/1) in a [demo bucket](https://console.cloud.google.com/storage/browser/ee-docs-demos/image-manifest). Follow [Cloud Storage instructions](https://cloud.google.com/storage/docs/uploading-objects#uploading-an-object) to upload your own.

## Build the manifest file

Build the manifest file from a dictionary.  Turn the dictionary into JSON.  Note the use of the `gsutil` tool to get a listing of files in a Cloud Storage bucket ([learn more about `gsutil`](https://cloud.google.com/storage/docs/gsutil)).  Also note that the structure of the manifest is described in detail [here](https://developers.google.com/earth-engine/image_manifest#manifest-structure-reference).  Because the data are categorical, a `MODE` pyramiding policy is specified.  Learn more about how Earth Engine builds image pyramids [here](https://developers.google.com/earth-engine/scale).

In [ ]:
# Specify the bucket where the source images are stored.
GCS_BUCKET = 'gs://ee-docs-demos/image-manifest'

# Specify the new asset's path (ensure you have project write permission).
ASSET_NAME = 'projects/YOUR-PROJECT/assets/ASSET-NAME'


# List the contents of the cloud folder.
cloud_files = !gsutil ls {GCS_BUCKET + '/*.tif'}

# Get the list of source URIs from the gsutil output.
sources_uris = [{'uris': [f]} for f in cloud_files]

# The enclosing object for the asset.
asset = {
  'name': ASSET_NAME,
  'tilesets': [
    {
      'sources': sources_uris
    }
  ],
  'bands': [
    {
      'id': 'cover_code',
      'pyramiding_policy': 'MODE',
      'missing_data': {
          'values': [0]
      }
    }
  ]
}

import json
print(json.dumps(asset, indent=2))

Inspect the printed JSON for errors.

## Upload to Earth Engine

If you are able to `cat` the written file, run the upload to Earth Engine. First, import the Earth Engine and geemap libraries and initialize to Earth Engine (authentication from `auth.authenticate_user()` above applies).

In [ ]:
import ee
import geemap

ee.Initialize(project='YOUR-PROJECT')

Start the ingestion task. It can take 15-20 minutes to complete and the asset size will be 255 MB. If you don't want to run the task, you can view the result in the next section using a publicly readable Google-stored copy of the asset.

In [ ]:
ee.data.startIngestion(ee.data.newTaskId()[0], asset)

## Visualize the uploaded image with geemap

Once the ingestion task is complete (check https://code.earthengine.google.com/tasks), visualize the asset using [geemap](https://geemap.org/). Here, we load the Google-stored asset, but you can edit the path to load yours if you ran the ingestion task.

In [ ]:
palette = [
    'a3ff73', # farmland, 10
    '267300', # forest, 20
    'ffff00', # grassland, 30
    '70a800', # shrub, 40
    '00ffff', # wetland, 50
    '005cff', # water, 60
    '004600', # tundra, 70
    'c500ff', # impervious, 80
    'ffaa00', # bare, 90
    'd1d1d1', # snow, ice, 100
]
vis = {'min': 10, 'max': 100, 'palette': palette}

# Google-stored asset for demo (optionally change to your copy).
ingested_image = ee.Image('projects/google/manifest_upload_demo')


m = geemap.Map()
m.center_object(ingested_image)
m.add_layer(ingested_image, vis, 'land cover')
m